# **Machine Learning for Neuroimaging 2024 — Assignment 3**

## **ABIDE I Functional Connectome Analysis**

You have been provided with a resting-state fMRI connectome dataset of 120 individuals diagnosed with Autism Spectrum Disorder (ASD) and 120 typical controls. Each connectome, i.e., each subject’s brain network and properties, is encoded as an 𝑁-by-𝑁 correlation matrix 𝐌, where $M_{i,j}$ is the functional connectivity (correlation in activation patterns) between region $i$ and region $j$.

Note that this is the preprocessed version of ABIDE provided by the Preprocessed Connectome Project (PCP).

For more information about this dataset's structure: http://preprocessed-connectomes-project.org/abide/

*Cameron Craddock, Yassine Benhajali, Carlton Chu, Francois Chouinard, Alan Evans, András Jakab, Budhachandra Singh Khundrakpam, John David Lewis, Qingyang Li, Michael Milham, Chaogan Yan, Pierre Bellec (2013). The Neuro Bureau Preprocessing Initiative: open sharing of preprocessed neuroimaging data and derivatives. In Neuroinformatics 2013, Stockholm, Sweden.*

**Relevant Libraries**
*  [NumPy](https://numpy.org/):https For numerical computing and handling multi-dimensional data.
*  [Pandas](https://pandas.pydata.org/): For structured data operations and manipulations.
*  [Matplotlib](https://matplotlib.org/): For creating static, interactive, and animated visualizations in Python.
*  [scikit-learn](https://scikit-learn.org/stable/): For implementing machine learning algorithms.
*  [nibabel](https://nipy.org/nibabel/): For reading and writing neuroimaging data formats.
*  [nilearn](https://nilearn.github.io/stable/index.html): For advanced neuroimaging data manipulation and visualization.
*  [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/): For writing and training Graph Neural Networks (GNNs).
*  [DGL (Deep Graph Library)](https://www.dgl.ai/): For deep learning on GNNs.

In [ ]:
%%capture
# @title Run to install needed packages.
!pip install nilearn torch torchvision torchaudio torch-geometric

In [ ]:
# @title Construct weighted graphs using our connectivity matrices
import numpy as np
import pickle as pkl
from nilearn.connectome import ConnectivityMeasure

'''
Load the connectivity matrices
Note: each connectivity matrix (sample) was derived using the NiftiLabelsMasker
and ConnectivityMeasure [with full Pearson's correlation] from nilearn and each
sample was also standardized by z-score
[i.e., zero mean scaled to unit variance w.r.t. sample std]
'''

data_path = './ABIDE_240.npz' # Be sure to update this to wherever you stored data

# Load the data
data = np.load(data_path)

# Accessing the arrays
connectomes = data['features']
labels_abide = data['labels']

print("ABIDE data loaded from NPZ file.")

# Check the number of subject functional scans fetched
print(f"Number of participants: {len(connectomes)}")

In [ ]:
from nilearn import plotting
from nilearn import datasets

# Retrieve AAL brain atlas for parcellation, more info here: https://www.sciencedirect.com/science/article/pii/S1053811901909784
parcellations = datasets.fetch_atlas_aal()
atlas_filename = parcellations.maps
labels_parc = parcellations.labels
print(f"Number of ROIs: {len(labels_parc)}")

## Q4.

Plot a random connectivity matrix from the dataset.

---

In [ ]:
cm_sample = connectomes[47]
print(f"Connectivity matrix shape: {cm_sample.shape}")

np.fill_diagonal(cm_sample, 0)

'''
Note that that each "pixel" in the image is a brain region of interest (ROI)
and the color denotes the strength (</>) and direction (+/-) of the correlation
 between any two ROIs.
'''
plotting.plot_matrix(cm_sample, figure=(6, 6),
                     labels=range(cm_sample.shape[-1]),
                     vmax=0.8, vmin=-0.8, reorder=False,
                     title=f'Correlation matrix for d={cm_sample.shape[0]}')

Plot glass brain plot of the `cm_sample`.



In [ ]:
# Grab center coordinates for atlas labels
coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_filename)

# plot glass brain with labels
plotting.plot_connectome(cm_sample,
                         coordinates,
                         edge_threshold="60%", # thresholded due to high density
                         title="AAL Atlas (func)"
                        )

The labels contain diagnostic group each participant is in. It is coded as:

*   1 = Autism Spectrum Disorder (ASD)
*   2 = Normal Control (NC)

Let's re-index this to `0=ASD` and `1=NC` due to zero-indexed systems required for ML and other data processing software.

In [ ]:
from collections import Counter

# Adjust labels to start from 0; 0=ASD, 1=NC
y_abide = labels_abide - 1

# Print class labels and counts
print("Class distribution:", [f"({key}: {value})" for key, value in Counter(y_abide).items()])

In [ ]:
# Separate connectomes into subgroups for any comparative analyses
connectomes_asd = connectomes[y_abide == 0]
connectomes_nc = connectomes[y_abide == 1]

Now, you're ready to use the data for machine learning or statistical analysis.

Note: If you are interested in using graph neural networks, feel free to use the below PyG starter code to create your custom PyG Dataset. Check the assignment description for links to related tutorials.

---

In [ ]:
# @title [GNNs ONLY] Create custom PyG ConnectomeDataset
import torch
from torch_geometric.data import Data, Dataset
from torch_geometric.utils import dense_to_sparse

class ConnectomeDataset(Dataset):
    def __init__(self, connectivity_matrices, labels, task="classification", transform=None, pre_transform=None):
        super(ConnectomeDataset, self).__init__(None, transform, pre_transform)
        self.connectivity_matrices = connectivity_matrices
        self.labels = labels
        self.task = task

    def len(self):
        return len(self.connectivity_matrices)

    def get(self, idx):
        # Convert the connectivity matrix to edge index and edge attributes
        connectivity_matrix = torch.tensor(self.connectivity_matrices[idx])
        edge_index, edge_attr = dense_to_sparse(connectivity_matrix)

        # Create a data object
        data = Data(edge_index=edge_index, edge_attr=edge_attr)
        data.x = connectivity_matrix.to(torch.float)
        if self.task == "classification":
          data.y = torch.tensor([self.labels[idx]], dtype=torch.long)
        else:
          data.y = torch.tensor([self.labels[idx]], dtype=torch.float)
        return data

### Example - GNN Classification

In [ ]:
import torch
import torch_geometric.transforms as T
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader

# Instantiate the dataset
abide_dataset = ConnectomeDataset(connectomes, y_abide)

loader = DataLoader(abide_dataset, batch_size=32, shuffle=True)

print(f'Number of graphs: {len(abide_dataset)}')
print("Number of node features: ", abide_dataset.num_node_features)
print(f'Number of edge features: {abide_dataset.num_edge_features}')

num_classes = 2

In [ ]:
# Define a Graph Neural Network model.
# For more tutorials and examples: https://pytorch-geometric.readthedocs.io/en/2.6.0/get_started/colabs.html

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(abide_dataset.num_node_features, hidden_channels)

        ###### Your code here ######
        ## 1. Add more convolutional layers and a final linear layer

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        ###### Your code here ######
        # 1. Obtain node embeddings
        x = ...


        # 2. Readout layer (using a global aggregation function e.g. global_mean_pool)
        x = ...

        # 3. Apply a final classifier (be sure to implement dropout with self.training)
        x = ...

        return F.log_softmax(x, dim=1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Implement your training function
def train(model, loader, optimizer, criterion, device):
    model.train()

    loss = 0
    ###### Your code here ######
    ## 1. Be sure to at least return the loss and accuracy.


    # Return the loss and accuracy
    return loss, accuracy

# Implement your test (evaluation) function
def test(model, loader, criterion, device):
    model.eval()

    loss = 0

    ###### Your code here ######
    ## 1. Return test loss, accuracy, precision, recall and F-1 score.


    # Return the loss, accuracy, precision, recall and F-1 score.
    return loss / len(loader), accuracy, precision, recall, f1

In [ ]:
import torch
from sklearn.model_selection import StratifiedKFold

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used.")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead.")

# Define the number of folds
num_folds = ...   ###### Your code here ######

# Define the number of epochs
num_epochs = ...   ###### Your code here ######

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)  # Set random_state for reproducibility

train_losses = []
train_accs = []
test_losses = []
test_accs = []
test_precs = []
test_recs = []
test_f1s = []

# Loop through the folds
for fold, (train_index, test_index) in enumerate(skf.split(connectomes, y_abide)):
    print(f"Fold {fold + 1}")

    # Create train and test datasets for the current fold
    train_dataset = torch.utils.data.Subset(abide_dataset, train_index)
    test_dataset = torch.utils.data.Subset(abide_dataset, test_index)

    # Create DataLoaders for the current fold
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # No need to shuffle test data

    # Instantiate your GNN model here
    model = GCN(hidden_channels=64).to(device)  # Replace with your model definition; move to GPU if available

    # Define optimizer and criterion
    optimizer = ...  ###### Your code here ######
    criterion = ...  ###### Your code here ######

    # Training loop
    for epoch in range(num_epochs):
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, ' +
        f'Train Accuracy: {train_acc:.4f}')

    # Evaluation on the test set
    test_loss, test_acc, test_prec, test_rec, test_f1 = test(model, test_loader, criterion, device)
    print(test_loss, test_acc, test_prec, test_rec, test_f1)
    print(f'Test | Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f} ' +
          f'Precision: {test_prec:.4f}, Recall: {test_rec:.4f} ' +
          f'F1: {test_f1:.4f}')

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    test_precs.append(test_prec)
    test_recs.append(test_rec)
    test_f1s.append(test_f1)

  # After training and evaluation, print the average results.
  ###### Your code here ######



## Q5.

### Example - Graph Theory Metrics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 'cm_sample' is a random connectome from our dataset
weights = cm_sample[np.triu_indices_from(cm_sample, k=1)]  # Extract upper triangle to avoid duplication

# Plotting the distribution of weights
plt.hist(weights, bins=30, alpha=0.7)
plt.title("Distribution of Edge Weights")
plt.xlabel("Weight (Correlation Value)")
plt.ylabel("Frequency")
plt.show()


### Example - Network Community Detection

In [ ]:
import networkx as nx
import numpy as np
import community.community_louvain as community_louvain
import matplotlib.pyplot as plt

from matplotlib import colormaps

'''Note: networkx expects non-negative edge weights for community detection;
convert connectome edge weights to absolute value (changes edge weight
information to strength only and not whether positively or negatively
correlated.)
'''
# Find the absolute maximum value in the connectome for scaling
abs_cm_sample = np.abs(cm_sample)

G = nx.from_numpy_array(abs_cm_sample)

# Use the Louvain method for community detection
partition = community_louvain.best_partition(G, weight='weight')

# Visualize the communities
pos = nx.spring_layout(G)  # Positioning of the nodes
cmap = colormaps['viridis']

for com in set(partition.values()):
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size=20,
                           node_color=[cmap(com / (max(partition.values()) + 1))])

nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()


### Example - Local Clustering Coefficient

In [ ]:
from scipy import stats

# Calculate local clustering coefficients for connectomes_asd
local_clustering_asd = []
for connectome in connectomes_asd:
    # Create a NetworkX graph from the correlation matrix
    G = nx.from_numpy_array(connectome)
    # Calculate local clustering coefficients for all nodes
    local_coeffs = nx.clustering(G)
    # Append the average local clustering coefficient to the list
    local_clustering_asd.append(np.mean(list(local_coeffs.values())))

# Calculate local clustering coefficients for connectomes_nc
local_clustering_nc = []
for connectome in connectomes_nc:
    # Create a NetworkX graph from the correlation matrix
    G = nx.from_numpy_array(connectome)
    # Calculate local clustering coefficients for all nodes
    local_coeffs = nx.clustering(G)
    # Append the average local clustering coefficient to the list
    local_clustering_nc.append(np.mean(list(local_coeffs.values())))

# Compare the average local clustering coefficients
avg_local_clustering_asd = np.mean(local_clustering_asd)
avg_local_clustering_nc = np.mean(local_clustering_nc)

print(f"Average Local Clustering Coefficient for ASD: {avg_local_clustering_asd:.4f}")
print(f"Average Local Clustering Coefficient for NC: {avg_local_clustering_nc:.4f}")

## Compare the subgroups
# Perform independent samples t-test
t_statistic, p_value = stats.ttest_ind(clustering_coeffs_asd, clustering_coeffs_nc)

print(f"T-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpret the results
if p_value < 0.05:
    print("There is a statistically significant difference in local clustering coefficients between ASD and NC.")
else:
    print("There is no statistically significant difference in local clustering coefficients between ASD and NC.")